In [1]:
import datetime
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate  # pip install tabulate

from qudi.util.network import netobtain
import importlib
sys.path.append('../xq1i')
import libxq1i
importlib.reload(libxq1i)

confocal2 = libxq1i.xq1i(pulsed_master_logic, pulsed_measurement_logic, sequence_generator_logic)

# ------------------------------------------------------------------
# 1) SELECT WHICH MEASUREMENTS TO PERFORM
# ------------------------------------------------------------------
selected_routines = {
    "Rabi": True,
    "ODMR": True,
    # For future expansions, e.g.:
    # "HahnEcho": False,
    # "XY8": False,
}

# ------------------------------------------------------------------
# 2) SETUP: Folder Creation, Orientation Props, POI List
# ------------------------------------------------------------------
results_root = "results"
timestamp_str = datetime.datetime.now().strftime('%Y%m%d_%H%M')
run_folder_name = f"characterization_run_{timestamp_str}"
run_folder = os.path.join(results_root, run_folder_name)
os.makedirs(run_folder, exist_ok=True)

# Dictionary to store all measurement results in tabulate-friendly form
all_results = []

orientation_properties = {
    1: {'res_freq': 2.705e9, 'freq_range': [2650.0e6, 5.0e6, 100],  'rabi_period': 3.0e-7},
    #2: {'res_freq': 2.665e9, 'freq_range': [2640.0e6, 1.0e6, 40],  'rabi_period': 5.06e-7},
    #3: {'res_freq': 2.840e9, 'freq_range': [2820.0e6, 1.0e6, 40],  'rabi_period': 2.93e-7},
    #4: {'res_freq': 2.887e9, 'freq_range': [2870.0e6, 1.0e6, 40],  'rabi_period': 2.05e-7},
    #5: {'res_freq': 2.900e9, 'freq_range': [2550.0e6, 2.5e6, 300], 'rabi_period': 2.05e-7},
}

poi_list = [
    #{'poi': 'POI_40', 'orientations': [2,4]},
    #{'poi': 'POI_54', 'orientations': [3,4]},
    #{'poi': 'POI_61', 'orientations': [1,2,3,4]},
    {'poi': 'POI1', 'orientations': [1]},
    {'poi': 'POI2', 'orientations': [1]},
    #{'poi': 'POI_3', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_4', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_5', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_6', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_7', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_8', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_9', 'orientations': [1,2,3,4]},
    #{'poi': 'POI_10', 'orientations': [1,2,3,4]},
]

# Clear pulse generator
pulsed_master_logic.clear_pulse_generator()
time.sleep(1)

# ------------------------------------------------------------------
# 3) MAIN LOOP OVER POIs AND ORIENTATIONS
# ------------------------------------------------------------------
for entry in poi_list:
    currPoi = entry['poi']
    print(f"Go to POI {currPoi} ...")

    # Disable refocus/tracking, move to POI, then re-enable
    poi_manager_logic.toggle_periodic_refocus(False)
    while (
        scanner_gui.module_state() == 'locked' or 
        (scanning_optimize_logic._sequence_index < len(scanning_optimize_logic._scan_sequence))
    ):
        time.sleep(0.1)

    poi_manager_logic.go_to_poi(currPoi)
    poi_manager_logic.set_active_poi(currPoi)
    poi_manager_logic.toggle_periodic_refocus(True)
    while (
        scanner_gui.module_state() == 'locked' or 
        (scanning_optimize_logic._sequence_index < len(scanning_optimize_logic._scan_sequence))
    ):
        time.sleep(0.1)

    print("Position and counts at peak after refocus:", scanner_gui.optimizer_dockwidget.result_label.text())
    for orientation in entry['orientations']:
        print(f"Start measurements for orientation {orientation} ...")
        # Take note of the start counts
        countsStart = scanner_gui.optimizer_dockwidget.result_label.text()[-9:]

        # Initialize placeholders for results

        rabi_period_val = None
        rabi_period_err = None
        rabi_contrast_val = None
        rabi_contrast_err = None
        rabi_rsquared = None
        rabi_redchi   = None

        odmr_center_val = None
        odmr_center_err = None
        odmr_sigma_val = None
        odmr_sigma_err = None
        odmr_contrast_val = None
        odmr_contrast_err = None
        odmr_rsquared = None
        odmr_redchi   = None

        # -------------------------
        # OPTIONAL ROUTINE: RABI
        # -------------------------
        if selected_routines.get("Rabi", False):
            confocal2.generate_params['microwave_frequency'] = orientation_properties[orientation]['res_freq']
            pulsed_master_logic.set_generation_parameters(confocal2.generate_params)
            confocal2.rabi_sweeps = 1e4

            rabi_label = f"{currPoi}_ornt_{orientation}_Rabi"
            print(f"Performing Rabi measurement: {rabi_label}")
            confocal2.do_rabi(rabi_label, isSlow=True)

            # Fit and extract results
            result_rabi = netobtain(pulsed_measurement_logic.do_fit('Sine'))

            # Retrieve best-fit values and errors
            amp_val    = result_rabi.params['amplitude'].value
            amp_err    = result_rabi.params['amplitude'].stderr
            off_val    = result_rabi.params['offset'].value
            off_err    = result_rabi.params['offset'].stderr
            rabi_period_val   = result_rabi.params['period'].value
            rabi_period_err   = result_rabi.params['period'].stderr

            rabi_rsquared = result_rabi.rsquared
            rabi_redchi   = result_rabi.redchi  # reduced chi-square


            # Rabi Contrast
            if off_val and amp_val is not None:
                contr_val = amp_val / off_val
                # Simple error propagation ignoring correlation
                if (amp_err is not None) and (off_err is not None) and (amp_val != 0) and (off_val != 0):
                    rel_err_sq = (amp_err / amp_val)**2 + (off_err / off_val)**2
                    contr_err = abs(contr_val) * np.sqrt(rel_err_sq)
                else:
                    contr_err = None
            else:
                contr_val = 0.0
                contr_err = None

            rabi_contrast_val = contr_val
            rabi_contrast_err = contr_err

            # Save raw Rabi data
            tData_rabi   = netobtain(pulsed_measurement_logic.signal_data[0])
            sigData_rabi = netobtain(pulsed_measurement_logic.signal_data[1])
            rabi_data_filename = os.path.join(run_folder, f"Rabi_{currPoi}_ornt_{orientation}.txt")
            with open(rabi_data_filename, 'w') as f:
                f.write("# time (s)\tcounts\n")
                for x, y in zip(tData_rabi, sigData_rabi):
                    f.write(f"{x}\t{y}\n")

            # Plot & save Rabi figure
            plt.figure(figsize=(7, 1.5))
            plt.plot(tData_rabi, sigData_rabi, 'o-', ms=3, linewidth=0.5, label='data')
            plt.plot(result_rabi.high_res_best_fit[0], result_rabi.high_res_best_fit[1], label='fit')
            plt.title(f"Rabi: T={rabi_period_val:.2E} s")
            plt.xlabel("time (s)")
            plt.ylabel("counts")
            plt.grid()
            plt.legend()
            rabi_fig_path = os.path.join(run_folder, f"Rabi_{currPoi}_ornt_{orientation}.png")
            plt.savefig(rabi_fig_path, dpi=150, bbox_inches='tight')
            plt.show()

        else:
            # If skipping Rabi, we might use a fallback rabi_period from orientation_props or just None
            rabi_period_val = orientation_properties[orientation]['rabi_period']
            # everything else remains None or "N/A"
            print("Skipping Rabi measurement.")

        # -------------------------
        # OPTIONAL ROUTINE: ODMR
        # -------------------------
        if selected_routines.get("ODMR", False):
            # Configure pulsed ODMR parameters
            props = orientation_properties[orientation]
            confocal2.pulsedODMR_params['freq_start']    = props['freq_range'][0]
            confocal2.pulsedODMR_params['freq_step']     = props['freq_range'][1]
            confocal2.pulsedODMR_params['num_of_points'] = props['freq_range'][2]

            # Use our measured/assumed Rabi period (could be None, but we just pass it in)
            confocal2.generate_params['rabi_period'] = rabi_period_val
            pulsed_master_logic.set_generation_parameters(confocal2.generate_params)
            confocal2.pulsedODMR_sweeps = 1e4

            odmr_label = f"{currPoi}_ornt_{orientation}_ODMR"
            print(f"Performing ODMR measurement: {odmr_label}")
            confocal2.do_pulsedODMR(odmr_label)

            result_odmr = netobtain(pulsed_measurement_logic.do_fit('Lorentzian Dip'))

            # Retrieve best-fit values & errors
            cen_val  = result_odmr.params['center'].value
            cen_err  = result_odmr.params['center'].stderr
            amp_val  = result_odmr.params['amplitude'].value
            amp_err  = result_odmr.params['amplitude'].stderr
            off_val  = result_odmr.params['offset'].value
            off_err  = result_odmr.params['offset'].stderr

            # Some Lorentzian fits have 'sigma' or 'gamma' or 'fwhm':
            # This assumes the fit used "sigma" as a parameter name.
            if 'sigma' in result_odmr.params:
                sig_val  = result_odmr.params['sigma'].value
                sig_err  = result_odmr.params['sigma'].stderr
            else:
                sig_val  = None
                sig_err  = None

            odmr_rsquared = result_odmr.rsquared
            odmr_redchi   = result_odmr.redchi

            odmr_center_val = cen_val
            odmr_center_err = cen_err
            odmr_sigma_val  = sig_val
            odmr_sigma_err  = sig_err

            # Contrast = -amp / offset
            if off_val and amp_val is not None:
                contr_val = -amp_val / off_val
                # Simple error propagation ignoring correlation
                if (amp_err is not None) and (off_err is not None) and (amp_val != 0) and (off_val != 0):
                    rel_err_sq = (amp_err / amp_val)**2 + (off_err / off_val)**2
                    contr_err = abs(contr_val) * np.sqrt(rel_err_sq)
                else:
                    contr_err = None
            else:
                contr_val = None
                contr_err = None

            odmr_contrast_val = contr_val
            odmr_contrast_err = contr_err

            # Wait for system to unlock
            while (
                scanner_gui.module_state() == 'locked' or 
                (scanning_optimize_logic._sequence_index < len(scanning_optimize_logic._scan_sequence))
            ):
                time.sleep(0.1)

            countsEnd = scanner_gui.optimizer_dockwidget.result_label.text()[-9:]
        else:
            print("Skipping ODMR measurement.")
            countsEnd = scanner_gui.optimizer_dockwidget.result_label.text()[-9:]

        # ------------------------------------------------------------------
        # 4) PACKAGE RESULTS FOR TABULATE (handling None => "N/A")
        # ------------------------------------------------------------------
        def fmt_or_na(val, fmt="{:.3E}"):
            """Helper: Format numerical value or return 'N/A' if None."""
            if val is None:
                return "N/A"
            try:
                return fmt.format(val)
            except:
                return str(val)

        row_data = {
            "POI":           currPoi,
            "Orientation":   orientation,
            "StartCounts":   countsStart,
            "EndCounts":     countsEnd,

            # --- Rabi Results ---
            "RabiPeriod":        fmt_or_na(rabi_period_val, "{:.4E}"),
            "RabiPeriodErr":     fmt_or_na(rabi_period_err, "{:.2E}"),
            "RabiContrast":      fmt_or_na(rabi_contrast_val, "{:.3f}"),
            "RabiContrastErr":   fmt_or_na(rabi_contrast_err, "{:.3f}"),
            "RabiRsq":           fmt_or_na(rabi_rsquared, "{:.4f}"),
            "RabiRedChi":        fmt_or_na(rabi_redchi, "{:.4f}"),

            # --- ODMR Results ---
            "ODMRFreq":          fmt_or_na(odmr_center_val, "{:.4E}"),
            "ODMRFreqErr":       fmt_or_na(odmr_center_err, "{:.2E}"),
            "ODMRSigma":         fmt_or_na(odmr_sigma_val, "{:.4E}"),
            "ODMRSigmaErr":      fmt_or_na(odmr_sigma_err, "{:.2E}"),
            "ODMRContrast":      fmt_or_na(odmr_contrast_val, "{:.3f}"),
            "ODMRContrastErr":   fmt_or_na(odmr_contrast_err, "{:.3f}"),
            "ODMRRsq":           fmt_or_na(odmr_rsquared, "{:.4f}"),
            "ODMRRedChi":        fmt_or_na(odmr_redchi, "{:.4f}"),
        }
        all_results.append(row_data)

print("All POIs measured successfully.")

# ------------------------------------------------------------------
# 5) CREATE SUMMARY TABLE
# ------------------------------------------------------------------
headers_map = {
    "POI":            "POI",
    "Orientation":    "Orientation",
    "StartCounts":    "Start Counts",
    "EndCounts":      "End Counts",

    "RabiPeriod":     "Rabi Period (s)",
    "RabiPeriodErr":  "Rabi PeriodErr",
    "RabiContrast":   "Rabi Contr",
    "RabiContrastErr":"Rabi ContrErr",
    "RabiRsq":        "Rabi Rsq",
    "RabiRedChi":     "Rabi RedChi",

    "ODMRFreq":       "ODMR Center (Hz)",
    "ODMRFreqErr":    "ODMR CenterErr",
    "ODMRSigma":      "ODMR Sigma (Hz)",
    "ODMRSigmaErr":   "ODMR SigmaErr",
    "ODMRContrast":   "ODMR Contr",
    "ODMRContrastErr":"ODMR ContrErr",
    "ODMRRsq":        "ODMR Rsq",
    "ODMRRedChi":     "ODMR RedChi",
}

table_str = tabulate(all_results, headers=headers_map, tablefmt="pretty")

summary_file_path = os.path.join(run_folder, "measurements_summary.txt")
with open(summary_file_path, 'w') as outputFile:
    outputFile.write(table_str + "\n")

print(table_str)


Go to POI POI1 ...


KeyboardInterrupt: 